# Convolutional Neural Network

### Importing the libraries

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')

In [4]:
tf.__version__

'2.17.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [7]:
# Preprocessing the Training set
# Applying image augmentation techniques to the training set, such as rescaling, shearing, zooming, and horizontal flipping
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
# Loading the training set from the directory with a target image size of 64x64 and a batch size of 32
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [9]:
# Preprocessing the Test set
# Only rescaling the test set images (no augmentation) 
test_datagen = ImageDataGenerator(rescale = 1./255)
# Loading the test set from the directory with a target image size of 64x64 and a batch size of 32
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [12]:
# Initialising the CNN
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [14]:
# Adding a convolutional layer with 32 filters, a 3x3 kernel size, ReLU activation, and an input shape of 64x64x3 (for RGB images)
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [16]:
# Adding a max pooling layer with a 2x2 pool size and a stride of 2 to reduce the spatial dimensions
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [18]:
# Adding a second convolutional layer with 32 filters and a 3x3 kernel, followed by another max pooling layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [20]:
# Flattening the pooled feature maps into a single vector
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [22]:
# Adding a fully connected dense layer with 128 units and ReLU activation
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [24]:
# Adding the output layer with a single unit and sigmoid activation for binary classification
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [27]:
# Using the Adam optimizer, binary crossentropy loss function, and accuracy as the evaluation metric
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [29]:
# Running the training process for 25 epochs with the training set and validating on the test set
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 22s 82ms/step - accuracy: 0.5472 - loss: 0.6870 - val_accuracy: 0.6575 - val_loss: 0.6376
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.6453 - loss: 0.6336 - val_accuracy: 0.7000 - val_loss: 0.5896
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.6806 - loss: 0.5983 - val_accuracy: 0.7100 - val_loss: 0.5748
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.7174 - loss: 0.5525 - val_accuracy: 0.7415 - val_loss: 0.5300
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - accuracy: 0.7277 - loss: 0.5299 - val_accuracy: 0.7615 - val_loss: 0.5145
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.7506 - loss: 0.5095 - val_accuracy: 0.7560 - val_loss: 0.5135
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - accuracy: 0.7729 - loss: 0.4770 - val_accuracy: 0.7690 - val_loss: 0.5055
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - accuracy: 0.7728 - loss: 0.4593 - 

## Part 4 - Making a single prediction

In [31]:
# Importing necessary libraries for making predictions
import numpy as np
from keras.preprocessing import image

# Loading a single image for prediction, resizing it to 64x64
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
# Converting the image to an array
test_image = image.img_to_array(test_image)
# Expanding dimensions to match the model's input shape
test_image = np.expand_dims(test_image, axis = 0)
# Predicting the class of the image (cat or dog)
result = cnn.predict(test_image)
# Getting the class indices to interpret the prediction
training_set.class_indices

# Interpreting the result to print whether the image is of a dog or a cat
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

# Printing the prediction
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
cat


In [32]:
print(prediction)

cat


## Making Multiple prediction

In [34]:
# Creating a list of file paths for images to be used for single predictions
location_list = ['dataset/single_prediction/cat_or_dog_1.jpg',
                'dataset/single_prediction/cat_or_dog_2.jpg',
                'dataset/single_prediction/cat_or_dog_3.jpg',
                'dataset/single_prediction/cat_or_dog_4.jpg',
                'dataset/single_prediction/cat_or_dog_5.jpg',
                'dataset/single_prediction/cat_or_dog_6.jpg',
                'dataset/single_prediction/cat_or_dog_7.jpg']

In [35]:
# Importing necessary libraries for image processing and predictions
import numpy as np
from keras.preprocessing import image

# Looping through each image location in the location_list
for img_loc in location_list:
    # Loading the image and resizing it to match the input shape required by the model
    test_image = image.load_img(img_loc, target_size = (64, 64))
    # Converting the image to an array
    test_image = image.img_to_array(test_image)
    # Expanding the dimensions of the array to match the model's expected input shape (adding batch dimension)
    test_image = np.expand_dims(test_image, axis = 0)
    # Predicting the class of the image (cat or dog)
    result = cnn.predict(test_image)
    # Accessing class indices to interpret the prediction
    training_set.class_indices
    # Interpreting the result: if the prediction is 1, it's a dog; otherwise, it's a cat
    if result[0][0] == 1:
        prediction = 'dog'
    else:
        prediction = 'cat'
    # Printing the prediction for the current image
    print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
dog
